# Building a Database for Crime Reports

## Introduction

In this project, we will build a PostgreSQL database for storing data related to crimes that occured in Boston. This dataset is available in the file `boston.csv`.

The file contains the following columns:

* **incident_number** - The unique ID of the crime
* **offense_code** - The numeric ID code for the committed crime classification
* **description** - A description of the crime
* **date** - The date that the crime occurred on
* **day_of_the_week** - The corresponding day of the week
* **lat** and **long** - The location of the crime

We will create a database named `crime_db` with a table - `boston_crimes` - with appropriate datatypes for storing the data from the `boston.csv` file. We will create this table within a schema named `crimes` to help organise our database.
We will also create two groups, `readonly` and `readwrite`, with the appropriate privileges and create one user for each group.

Here is a high-level overview of what we wish to build:

![overview](images/overview.png)

## Creating the Crime Database

Let's start by connecting to the `postgres` superuser and creating the `crime_db` database:


In [1]:
import psycopg2

conn = psycopg2.connect(user='postgres', password='secret')
conn.autocommit = True
cur = conn.cursor()

# Drop database if it already exists to allow us to re-run the kernel.
cur.execute('DROP DATABASE crime_db;')

cur.execute('CREATE DATABASE crime_db;') 

conn.autocommit = False

conn.commit()
conn.close()


Now we can connect to our new `crime_db` database and keep the connection open throughout the project. Let's also create our schema, `crimes`, to allow us to organise our tables better.

In [2]:
conn = psycopg2.connect(dbname='crime_db', user='postgres', password='secret')
cur = conn.cursor()

cur.execute('CREATE SCHEMA crimes;')

conn.commit()

## Obtaining the Column Names and Sample

Before we create our tables, it will help to get an idea of what the data in the crime dataset looks like. We will then be able to easily select the right datatypes to use.

Let's start by reading the column names from the `boston.csv` file as well as the first row. This way, we will have them at hand throughout the project for reference.

In [3]:
import csv

with open('boston.csv', mode='r') as file:
    rows = list(csv.reader(file))
    col_headers = rows[0]
    first_row = rows[1]

print(col_headers)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


We now have access to the column headers and the first row of data. This will help us when we create the table. We won't have to remember the names of the columns, and using the first row of data, we can easily recall what kind of data is in each column.

## Creating an Auxiliary Function

Before we create a table to store the crime data, we need to identify the proper datatypes for the columns. To help us with that, let's create a function - `get_col_set()` - that, given the name of a CSV file and a column index, computes a Python set with all distinct values contained in that column.

This function will be useful for two reasons:

1. Checking whether an enumerated datatype might be a good choice for representing a column
2. Computing the maximum length of any text-like column to select appropriate sizes for `VARCHAR` columns. The same logic applies for the maximum values of numerical columns.


In [4]:
# Create a get_col_set() function that returns a set of distinct values for a column.
def get_col_set(csv_filename: str, col_index: int):
    with open(csv_filename, mode='r') as file:
        next(file) # Skip the header row.
        reader = csv.reader(file)
        rows = [row for row in reader]
    
    col_set = set()
    for row in rows:
        col_set.add(row[col_index])
        
    return col_set
    
# Use the get_col_set() function to calculate the number of distinct values in each column of the crime dataset.
col_num_elements = {}
for col_index in range(len(col_headers)):
    col_elements = get_col_set('boston.csv', col_index)
    num_elements = len(col_elements)
    col_num_elements[col_index] = num_elements
    
print(col_num_elements)

{0: 298329, 1: 219, 2: 239, 3: 1177, 4: 7, 5: 18177, 6: 18177}


We can see here that the `day_of_the_week` column only has 7 distinct values, which matches what we would expect. Therefore, we will make this column have a enumerated datatype.

## Finding the Maximum Length

Another important aspect when choosing datatypes is to know the longest string in any column containing textual data.

As we already know that the longest string in the `day_of_the_week` column is `Wednesday`, let's just calculate the longest string in the `description` column.

In [5]:
#find the index for the description column.
print(col_headers)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


In [6]:
# Compute the maximum length of any value in the description column.
description_elements = get_col_set('boston.csv', 2)
element_lengths = set()
for element in description_elements:
    element_lengths.add(len(element))

max_length = max(element_lengths)
print(max_length)

58


## Creating the Table

Next, we'll create a table named `boston_crimes` inside the `crimes` schema of the `crime_db` database. But first, we'll use the information gathered from the two previous sections to select the appropriate datatypes.

In [7]:
print(col_headers)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


We'll use the headers in the `boston.csv` file as our column names for simplicity, as they will be familiar to the users.

In [8]:
print(first_row)

['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


If we look at the data within the first row of the dataset, we can see the following data types:

Integer Numeric
Integer Numeric
String
Date
String
Decimal Numeric
Decimal Numeric

We'll set column data types that work well with the data. Also, recall that the `day_of_the_week` column only had 7 distinct values, which means it is suited to have an enumerated datatype.We'll create that datatype before creating the table.


In [9]:
# Create enumerated datatype for day_of_the_week column.
cur.execute('''
            CREATE TYPE day_enum AS ENUM (
                'Monday',
                'Tuesday',
                'Wednesday',
                'Thursday',
                'Friday',
                'Saturday',
                'Sunday'
            );
            '''
            )

# Create the boston_crimes table within the crimes schema.
cur.execute('''
            CREATE TABLE crimes.boston_crimes (
                incident_number INTEGER PRIMARY KEY,
                offense_code INTEGER,
                description VARCHAR(100),
                date DATE,
                day_of_the_week day_enum,
                lat DECIMAL,
                long DECIMAL
            );
            '''
           )

conn.commit()

# Call the cursor.description attribute to ensure table has been created correctly.
cur.execute('SELECT * FROM crimes.boston_crimes LIMIT 0;')
print(cur.description)

(Column(name='incident_number', type_code=23), Column(name='offense_code', type_code=23), Column(name='description', type_code=1043), Column(name='date', type_code=1082), Column(name='day_of_the_week', type_code=16710), Column(name='lat', type_code=1700), Column(name='long', type_code=1700))


## Loading the Data

Now that we have created the table, we can load the data into it. As we are loading the data from a CSV file, which may have commas hidden in the string data, we'll use the cursor.copy_expert() method. This will ensure the file is interpreted as a CSV and parsed accordingly.

In [10]:
# Load the data into the boston_crimes table.
with open('boston.csv', mode='r') as file:
    cur.copy_expert('COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER', file)
    
conn.commit()

# Print first few rows of the table.
cur.execute('''
            SELECT *
            FROM crimes.boston_crimes
            LIMIT 3;
            ''')

sample = cur.fetchall()
print(sample)

[(1, 619, 'LARCENY ALL OTHERS', datetime.date(2018, 9, 2), 'Sunday', Decimal('42.35779134'), Decimal('-71.13937053')), (2, 1402, 'VANDALISM', datetime.date(2018, 8, 21), 'Tuesday', Decimal('42.30682138'), Decimal('-71.06030035')), (3, 3410, 'TOWED MOTOR VEHICLE', datetime.date(2018, 9, 3), 'Monday', Decimal('42.34658879'), Decimal('-71.07242943'))]


After this step, our database now looks like this:

![table_created](images/table_created.png)

## Revoking Public Privileges

Let's recap our progress so far: we have created a database with a schema inside it to hold all data about crimes; we selected the most appropriate datatypes for storing the data and created a table; and we loaded the CSV contaiming crimes about Boston into the table.

Now, it's time to handle users. Our goal is to create the two common user groups `readonly` (can only SELECT data) and `readwrite` (can SELECT data as well as INSERT, UPDATE and DELETE).

If we are to follow the least privilege principle, we must ensure that each user only has the privileges that are absolutely necessary for their role.
The first step in doing so is to make sure that there are no privileges inherited from the `public` group and on the `public` schema.

In [11]:
# Revoke all privileges of the public group on tables within the public schema.
cur.execute('REVOKE ALL ON SCHEMA public FROM public;')

# Revoke all privileges of the public group on the crime_db database.
cur.execute('REVOKE ALL ON DATABASE crime_db FROM public')

conn.commit()

## Creating User Groups

Now we've made sure that we aren't going to inadvertently inherit privilages from the `public` group, our next step is to create our two user groups:

* `readonly` - only has privileges to perform `SELECT` queries
* `readwrite` - has privileges to perform `SELECT`, `INSERT`, `DELETE` and `UPDATE`

Both of our users will also need connection privileges on the `crime_db` database and usage of the `crimes` schema.

In [12]:
# Drop groups if they already exist to allow us to re-run kernel.
cur.execute('DROP GROUP readonly;')
cur.execute('DROP GROUP readwrite;')

# Create the two groups with the NOLOGIN option.
cur.execute('CREATE GROUP readonly NOLOGIN;')
cur.execute('CREATE GROUP readwrite NOLOGIN;')

# Grant connection privilages to both groups.
cur.execute('GRANT CONNECT ON DATABASE crime_db TO readonly;')
cur.execute('GRANT CONNECT ON DATABASE crime_db TO readwrite;')

# Grant usage of the crimes schema to both groups.
cur.execute('GRANT USAGE ON SCHEMA crimes TO readonly;')
cur.execute('GRANT USAGE ON SCHEMA crimes TO readwrite;')

# Grant group specific privileges to all tables in the crimes schema.
cur.execute('GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;')
cur.execute('GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite;')

conn.commit()

Right now, the high-level picture of our database looks like this:

![user_groups](images/user_groups.png)

## Creating Users

The final thing we need to do in order for our database to be configured as planned is to create each user and then assign them to each group.

In [13]:
# Drop users if they already exist to allow us to re-run kernel.
cur.execute('DROP USER data_analyst;')
cur.execute('DROP USER data_scientist;')

# Create data_analyst user and assign them to the readonly group.
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")
cur.execute('GRANT readonly TO data_analyst;')

# Create data_scientist user and assign them to the readwrite group.
cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")
cur.execute('GRANT readwrite TO data_scientist;')

conn.commit()

## Testing

Now that we've finished creating our database, tables, groups and users, it's important to check that they have all been set up correctly.

We can query the internal `pg_roles` table to inspect user privileges related to the database and the `information_schema.table_privileges` table to inspect table privileges.

We've already checked the `boston_crimes` table to see that it has been set up correctly.

In [14]:
# Check the database privileges for data_analyst and data_scientist users.
cur.execute('''
            SELECT
              rolname,
              rolsuper,
              rolcreaterole,
              rolcreatedb
            FROM pg_roles
            WHERE rolname = 'data_analyst'
              OR rolname = 'data_scientist';
            '''
           )
db_privileges = cur.fetchall()
print(db_privileges)

[('data_analyst', False, False, False), ('data_scientist', False, False, False)]


Here, we can see that the `data_analyst` and `data_scientist` users are not superusers and cannot create new users or databases. This is what we intended.

In [15]:
# Check the table privileges for data_analyst and data_scientist users.
cur.execute('''
            SELECT
              grantee,
              table_name,
              privilege_type
            FROM information_schema.table_privileges
            WHERE table_name = 'boston_crimes'
              AND (grantee = 'readonly' OR grantee = 'readwrite')
            ORDER BY grantee;
            '''
           )
table_privileges = cur.fetchall()
print(table_privileges)

[('readonly', 'boston_crimes', 'SELECT'), ('readwrite', 'boston_crimes', 'DELETE'), ('readwrite', 'boston_crimes', 'UPDATE'), ('readwrite', 'boston_crimes', 'SELECT'), ('readwrite', 'boston_crimes', 'INSERT')]


We can also see that the users have the correct permissions in relation to the `boston_crimes` table.

The database is set up as intended and at a high-level, looks like this:

![overview](images/overview.png)